In [1]:
import os
import seaborn as sns
import nibabel as nib
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt

In [2]:
def softmax(x, T):
    x = x / T
    f = np.exp(x - np.max(x, axis = 0))  # shift values
    return f / f.sum(axis = 0)

def ComputMetric(ACTUAL, PREDICTED):
    ACTUAL = ACTUAL.flatten()
    PREDICTED = PREDICTED.flatten()
    idxp = ACTUAL == True
    idxn = ACTUAL == False

    tp = np.sum(ACTUAL[idxp] == PREDICTED[idxp])
    tn = np.sum(ACTUAL[idxn] == PREDICTED[idxn])
    fp = np.sum(idxn) - tn
    fn = np.sum(idxp) - tp
    FPR = fp / (fp + tn)
    if tp == 0 :
        dice = 0
        Precision = 0
        Sensitivity = 0
    else:
        dice = 2 * tp / (2 * tp + fp + fn)
        Precision = tp / (tp + fp)
        Sensitivity = tp / (tp + fn)
    return dice, Sensitivity, Precision

In [3]:
def sum_tensor(inp, axes, keepdim=False):
    axes = np.unique(axes).astype(int)
    if keepdim:
        for ax in axes:
            inp = inp.sum(int(ax), keepdim=True)
    else:
        for ax in sorted(axes, reverse=True):
            inp = inp.sum(int(ax))
    return inp

def get_tp_fp_fn(net_output, gt, axes=None, mask=None, square=False):
    """
    net_output must be (b, c, x, y(, z)))
    gt must be a label map (shape (b, 1, x, y(, z)) OR shape (b, x, y(, z))) or one hot encoding (b, c, x, y(, z))
    if mask is provided it must have shape (b, 1, x, y(, z)))
    :param net_output:
    :param gt:
    :param axes:
    :param mask: mask must be 1 for valid pixels and 0 for invalid pixels
    :param square: if True then fp, tp and fn will be squared before summation
    :return:
    """
    if axes is None:
        axes = tuple(range(2, len(net_output.size())))

    shp_x = net_output.shape
    shp_y = gt.shape

    with torch.no_grad():
        if len(shp_x) != len(shp_y):
            gt = gt.view((shp_y[0], 1, *shp_y[1:]))

        if all([i == j for i, j in zip(net_output.shape, gt.shape)]):
            # if this is the case then gt is probably already a one hot encoding
            y_onehot = gt
        else:
            gt = gt.long()
            y_onehot = torch.zeros(shp_x)
            if net_output.device.type == "cuda":
                y_onehot = y_onehot.cuda(net_output.device.index)
            y_onehot.scatter_(1, gt, 1)

    tp = net_output * y_onehot
    fp = net_output * (1 - y_onehot)
    fn = (1 - net_output) * y_onehot

    if mask is not None:
        tp = torch.stack(tuple(x_i * mask[:, 0] for x_i in torch.unbind(tp, dim=1)), dim=1)
        fp = torch.stack(tuple(x_i * mask[:, 0] for x_i in torch.unbind(fp, dim=1)), dim=1)
        fn = torch.stack(tuple(x_i * mask[:, 0] for x_i in torch.unbind(fn, dim=1)), dim=1)

    if square:
        tp = tp ** 2
        fp = fp ** 2
        fn = fn ** 2

    tp = sum_tensor(tp, axes, keepdim=False)
    fp = sum_tensor(fp, axes, keepdim=False)
    fn = sum_tensor(fn, axes, keepdim=False)

    return tp, fp, fn


def SoftDiceLoss(x, y, loss_mask = None, smooth = 1e-5, batch_dice = False):
    '''
    Batch_dice means that we want to calculate the dsc of all batch
    It would make more sense for small patchsize, aka DeepMedic based training.
    '''
    shp_x = x.shape
    square = False

    axes = [0] + list(range(2, len(shp_x)))
    tp, fp, fn = get_tp_fp_fn(x, y, axes, loss_mask, square)
    dc = (2 * tp + smooth) / (2 * tp + fp + fn + smooth)
    dc_process = dc

    return dc_process

In [4]:
resultpath = '/vol/biomedic3/zl9518/ModelEvaluation/output/prostate/prostateval/'
prostatevalpath = '/vol/biomedic3/zl9518/Prostatedata/datafiletestB/'

In [5]:
DatafiletsImgc1 = prostatevalpath + 'seg-eval.txt'
Imgfiletsc1 = open(DatafiletsImgc1)
Imgreadc1 = Imgfiletsc1.read().splitlines()

In [6]:
probs_all = []
preds_class_all = []
targets_all = []
for Imgnamec1 in tqdm(Imgreadc1):
    knamelist = Imgnamec1.split("/")
    kname = knamelist[-1][0:6]
    
    cls0filename = resultpath + '/results/pred_' + kname + 'cls0_prob.nii.gz'
    cls1filename = resultpath + '/results/pred_' + kname + 'cls1_prob.nii.gz'
    cls0read = nib.load(cls0filename)
    cls1read = nib.load(cls1filename)
    cls0logit = cls0read.get_fdata()
    cls1logit = cls1read.get_fdata()
    GTread = nib.load(Imgnamec1)
    GTimg = GTread.get_fdata()
    imgshape = GTimg.shape
    cls0flatten = cls0logit.flatten()
    cls1flatten = cls1logit.flatten()
    clsflatten = np.stack((cls0flatten, cls1flatten))
    GTflatten = GTimg.flatten()
    probflatten = softmax(clsflatten, T = 1.0)
    
    pred_class = np.argmax(probflatten, axis = 0)
    preds_class_all = np.concatenate((preds_class_all, pred_class), axis=0)
    targets_all = np.concatenate((targets_all, GTflatten), axis=0)
    probmax = np.max(probflatten, axis = 0)
    probs_all = np.concatenate((probs_all, probmax), axis=0)


100%|█████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.50it/s]


In [7]:
threshold = 0.877
print(np.sum(targets_all == preds_class_all) / len(targets_all))
print(np.sum(probs_all > threshold) / len(targets_all))

0.9960332029006206
0.9960383582999901


In [16]:
# probs_all_c0 = []
# preds_class_all_c0 = []
# targets_all_c0 = []
# probs_all_c1 = []
# preds_class_all_c1 = []
# targets_all_c1 = []
softDSCs = []
realDSCs = []
for Imgnamec1 in tqdm(Imgreadc1):
    knamelist = Imgnamec1.split("/")
    kname = knamelist[-1][0:6]
    
    cls0filename = resultpath + '/results/pred_' + kname + 'cls0_prob.nii.gz'
    cls1filename = resultpath + '/results/pred_' + kname + 'cls1_prob.nii.gz'
    cls0read = nib.load(cls0filename)
    cls1read = nib.load(cls1filename)
    cls0logit = cls0read.get_fdata()
    cls1logit = cls1read.get_fdata()
    GTread = nib.load(Imgnamec1)
    GTimg = GTread.get_fdata()
    imgshape = GTimg.shape
    cls0flatten = cls0logit.flatten()
    cls1flatten = cls1logit.flatten()
    clsflatten = np.stack((cls0flatten, cls1flatten))
    GTflatten = GTimg.flatten()
    
    preds_all_argmax = np.argmax(clsflatten, axis = 0)
    # for cls 0, BG class 
#     targets_y1 = np.where(preds_all_argmax==0)[0]
#     probflatten_c0 = softmax(clsflatten[:, targets_y1], T = 1.0)
#     pred_class_c0 = np.argmax(probflatten_c0, axis = 0)
#     preds_class_all_c0 = np.concatenate((preds_class_all_c0, pred_class_c0), axis=0)
    
#     targets_all_c0 = np.concatenate((targets_all_c0, GTflatten[targets_y1]), axis=0)
#     probmax_c0 = np.max(probflatten_c0, axis = 0)
#     probs_all_c0 = np.concatenate((probs_all_c0, probmax_c0), axis=0)
    
#     # for cls 1, FG class
#     targets_y1 = np.where(preds_all_argmax==1)[0]
#     probflatten_c1 = softmax(clsflatten[:, targets_y1], T = 1.0)
#     pred_class_c1 = np.argmax(probflatten_c1, axis = 0)
#     preds_class_all_c1 = np.concatenate((preds_class_all_c1, pred_class_c1), axis=0)
    
#     targets_all_c1 = np.concatenate((targets_all_c1, GTflatten[targets_y1]), axis=0)
#     probmax_c1 = np.max(probflatten_c1, axis = 0)
#     probs_all_c1 = np.concatenate((probs_all_c1, probmax_c1), axis=0)

    # for cls 0, BG class 
    probflatten = softmax(clsflatten, T = 1.0)
    probflattens = probflatten
    targets_y1 = np.where(preds_all_argmax==0)[0]
    probflatten[:, targets_y1] = softmax(clsflatten[:, targets_y1], T = 1.0)
    threshold = 0.857
    probflattens[:, targets_y1] = probflatten[:, targets_y1] > threshold
    probflattens = probflattens.astype(float)
    # for cls 1, FG class
    targets_y1 = np.where(preds_all_argmax==1)[0]
    probflatten[:, targets_y1] = softmax(clsflatten[:, targets_y1], T = 1.0)
    threshold = 0.970
    probflattens[:, targets_y1] = probflatten[:, targets_y1] > threshold
    probflattens = probflattens.astype(float)
    probr = probflatten.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
    probrs = probflattens.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
    probr_tensor = torch.tensor(probrs[np.newaxis, ...])
    GTimgf = np.argmax(probr, axis = 0)
    GT_tensor = torch.tensor(GTimgf[np.newaxis, ...])
    
    softDSC = SoftDiceLoss(probr_tensor, GT_tensor)
    softDSCs.append(softDSC[1].numpy())

    realDSC, _, _ = ComputMetric(GTimg, np.argmax(probr, axis = 0))
    realDSCs.append(realDSC)z


100%|█████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.05it/s]


In [17]:
# print(np.sum(targets_all_c0 == preds_class_all_c0) / len(targets_all_c0))
# threshold = 0.857
# print(np.sum(probs_all_c0 > threshold) / len(targets_all_c0))
# print(np.sum(targets_all_c1 == preds_class_all_c1) / len(targets_all_c1))
# threshold = 0.903
# print(np.sum(probs_all_c1 > threshold) / len(targets_all_c1))
print(np.mean(softDSCs))
print(np.mean(realDSCs))

0.9980038587250382
0.9979738236461122


In [107]:
# 84 conditions
softDSCs_AC = []
softDSCs_ATC = []
softDSCs_CSATC = []
realDSCs = []
for kcon in tqdm(range(1, 84)):
    softDSC_FG_AC = []
    softDSC_FG_ATC = []
    softDSC_FG_CSATC = []
    realDSC_FG = []
    resultpath = '/vol/biomedic3/zl9518/ModelEvaluation/output/prostate/prostattestcondition_' + str(kcon) + '/'
    for Imgnamec1 in Imgreadc1:
        knamelist = Imgnamec1.split("/")
        kname = knamelist[-1][0:6]

        cls0filename = resultpath + '/results/pred_' + kname + 'cls0_prob.nii.gz'
        cls1filename = resultpath + '/results/pred_' + kname + 'cls1_prob.nii.gz'
        cls0read = nib.load(cls0filename)
        cls1read = nib.load(cls1filename)
        cls0logit = cls0read.get_fdata()
        cls1logit = cls1read.get_fdata()
        GTread = nib.load(Imgnamec1)
        GTimg = GTread.get_fdata()
        imgshape = GTimg.shape
        cls0flatten = cls0logit.flatten()
        cls1flatten = cls1logit.flatten()
        clsflatten = np.stack((cls0flatten, cls1flatten))
        GTflatten = GTimg.flatten()
        # By AC
        probflatten = softmax(clsflatten, T = 1.0)
        probr = probflatten.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
        probr_tensor = torch.tensor(probr[np.newaxis, ...])
        GTimgf = np.argmax(probr, axis = 0)
        GT_tensor = torch.tensor(GTimgf[np.newaxis, ...])
        softDSC = SoftDiceLoss(probr_tensor, GT_tensor)
        softDSC_FG_AC.append(softDSC[1].numpy())
        # By ATC
        probflatten = softmax(clsflatten, T = 1.0)
        threshold = 0.877
        probflattens = probflatten > threshold
        probflattens = probflattens.astype(float)
        
        probr = probflatten.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
        GTimgf = np.argmax(probr, axis = 0)
        GT_tensor = torch.tensor(GTimgf[np.newaxis, ...])
        probrs = probflattens.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
        probr_tensor = torch.tensor(probrs[np.newaxis, ...])
        softDSC = SoftDiceLoss(probr_tensor, GT_tensor)
        softDSC_FG_ATC.append(softDSC[1].numpy())
        # By CSATC
        preds_all_argmax = np.argmax(clsflatten, axis = 0)
        # for cls 0, BG class 
        targets_y1 = np.where(preds_all_argmax==0)[0]
        probflatten[:, targets_y1] = softmax(clsflatten[:, targets_y1], T = 1.0)
        threshold = 0.857
        probflattens[:, targets_y1] = probflatten[:, targets_y1] > threshold
        # for cls 1, FG class
        targets_y1 = np.where(preds_all_argmax==1)[0]
        probflatten[:, targets_y1] = softmax(clsflatten[:, targets_y1], T = 1.0)
        threshold = 0.970
        probflattens[:, targets_y1] = probflatten[:, targets_y1] > threshold
        probflattens = probflattens.astype(float)
        probr = probflatten.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
        probrs = probflattens.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
        probr_tensor = torch.tensor(probrs[np.newaxis, ...])
        GTimgf = np.argmax(probr, axis = 0)
        GT_tensor = torch.tensor(GTimgf[np.newaxis, ...])
        softDSC = SoftDiceLoss(probr_tensor, GT_tensor)
        softDSC_FG_CSATC.append(softDSC[1].numpy())

        realDSC, _, _ = ComputMetric(GTimg, np.argmax(probr, axis = 0))
        realDSC_FG.append(realDSC)
        
    softDSCs_AC.append(np.mean(np.array(softDSC_FG_AC)))
    softDSCs_ATC.append(np.mean(np.array(softDSC_FG_ATC)))
    softDSCs_CSATC.append(np.mean(np.array(softDSC_FG_CSATC)))
    realDSCs.append(np.mean(np.array(realDSC_FG)))

100%|███████████████████| 83/83 [18:27<00:00, 13.35s/it]


In [117]:
print('AC_results:')
print(np.mean(np.abs(np.array(realDSCs) - np.array(softDSCs_AC))))
print('ATC_results:')
print(np.mean(np.abs(np.array(realDSCs) - np.array(softDSCs_ATC))))
print('CSATC_results:')
print(np.mean(np.abs(np.array(realDSCs) - np.array(softDSCs_CSATC))))

AC_results:
0.08682389850061455
ATC_results:
0.08557026647305374
CSATC_results:
0.01134276768333814


In [118]:
print(realDSCs)

[0.8457301025307677, 0.870852084028978, 0.828521585768393, 0.790703746725564, 0.7586812862096527, 0.7557484783931, 0.8172327417988589, 0.6604643235449457, 0.8391672071049449, 0.629645911710675, 0.8541350026276865, 0.8736883469998509, 0.8538392221470215, 0.843655497285545, 0.8472068280655369, 0.8415826376847552, 0.8719698924620335, 0.8669337478504607, 0.8188883956186596, 0.8289616381345647, 0.7512113795312175, 0.7984512037339606, 0.8258757147417111, 0.7808367447866433, 0.8216387475687809, 0.7407959047695183, 0.8082450924634974, 0.7488012437878321, 0.8813964100757584, 0.8449041600923041, 0.8723101712065781, 0.7438437175087971, 0.7690050430372952, 0.8036237249343758, 0.6054180714311662, 0.594775255065044, 0.8655259284098629, 0.8193493642947048, 0.8471036691597765, 0.8195559172815617, 0.8942332191332024, 0.886877812107558, 0.8949294033819303, 0.8655104773584853, 0.8947390346757661, 0.8509641068080747, 0.8916376091423593, 0.8932068281575198, 0.8886573243066662, 0.894043014237129, 0.88414099

In [119]:
print(softDSCs_CSATC)

[0.8573098450600742, 0.8600617529258539, 0.8511011800465511, 0.7610844443132836, 0.804225260350873, 0.6994625775600914, 0.8343077680867872, 0.5873883511877259, 0.8706367818162295, 0.5339708737304262, 0.8609286835520462, 0.8745588154827783, 0.8403775389417912, 0.8307831808346533, 0.8268898561923944, 0.8178379643873039, 0.8766403422994316, 0.8773801044190288, 0.8150702273271563, 0.8208535644503716, 0.7608286257043855, 0.7744383555214596, 0.8241560744704524, 0.7945163828758084, 0.8251698745952046, 0.7650909563272303, 0.8033009616996422, 0.775191699411599, 0.8813952990882676, 0.8511207214924168, 0.8724475501828607, 0.7128424443954274, 0.7445548932485803, 0.8172331937923939, 0.6486932954872577, 0.6261100425329447, 0.8695209492318604, 0.8071358207942033, 0.8268769963475917, 0.8375713993517863, 0.8934472807140402, 0.8847814183850069, 0.8943232197369081, 0.8623928516691762, 0.8936365726907469, 0.850068732989435, 0.890262573033971, 0.8920501421527687, 0.8869467539993081, 0.892964061399845, 0.88

In [120]:
print(softDSCs_ATC)

[0.9327430434040498, 0.9378185544854519, 0.9235938340434025, 0.8887076731960827, 0.8980355965887018, 0.8623144028533967, 0.9139631172921199, 0.7935959157041663, 0.9359708308937295, 0.7627235374306184, 0.9338440006464964, 0.9438774865471279, 0.9277106442572324, 0.9215330720071491, 0.9192820047470296, 0.910710004457669, 0.945640048371857, 0.9463451534286573, 0.9144148581126073, 0.9157026992753277, 0.8883251761930238, 0.8918552898827743, 0.9164302206070701, 0.9030024851692705, 0.9148374331392123, 0.8868819000455664, 0.9009367099490613, 0.8923107859810571, 0.950903975491976, 0.9353683840105426, 0.9482715857870654, 0.8543321439640149, 0.869384858425738, 0.9253469251009279, 0.834054241217984, 0.8412640539162384, 0.946280918471361, 0.9181433266337031, 0.9303256093728377, 0.9330631718014264, 0.9581101680541646, 0.950782862848264, 0.9591669752540989, 0.9377284310228851, 0.959041002108993, 0.9326961557549968, 0.9550094581183686, 0.9566148152600666, 0.9522982678392369, 0.9574968827574347, 0.94878

In [121]:
print(softDSCs_AC)

[0.9445208141411179, 0.5600351926555271, 0.931730127079436, 0.6528295465927425, 0.9158539060827395, 0.7136134168357384, 0.9246074382786308, 0.6644763539333387, 0.9389497001221697, 0.7066098338452236, 0.9275612026510706, 0.9325569574687153, 0.9328646004932912, 0.9269801658772193, 0.9270234060337872, 0.9238830251708064, 0.9326770866378464, 0.9336464948378751, 0.9172174173486736, 0.9205507685038764, 0.9035785475003083, 0.9033867852464283, 0.7677991683602265, 0.77648857500991, 0.7670021114181449, 0.7822310609348627, 0.7681708070921458, 0.7782648221080334, 0.9547345456520384, 0.9456113594412106, 0.9535450027183474, 0.9004201920300152, 0.8979303404342277, 0.9448680173310867, 0.8859718451761012, 0.8879338421392813, 0.9540584116065987, 0.9288409827503431, 0.9366174510538441, 0.9459088235601767, 0.9619089739592205, 0.9532975605133911, 0.9635967070577143, 0.9466395431589174, 0.9636146188932045, 0.9455708263766514, 0.9578893452463093, 0.9599190227327112, 0.9549463403474387, 0.961062565393781, 0.9

In [122]:
# 5 conditions
domainlist = ['A', 'C', 'D', 'E', 'F']
softDSCs_AC = []
softDSCs_ATC = []
softDSCs_CSATC = []
realDSCs = []
for kcon in tqdm(domainlist):
    softDSC_FG_AC = []
    softDSC_FG_ATC = []
    softDSC_FG_CSATC = []
    realDSC_FG = []
    resultpath = '/vol/biomedic3/zl9518/ModelEvaluation/output/prostate/prostattestcondition_' + kcon + '/'
    
    prostatevalpath = '/vol/biomedic3/zl9518/Prostatedata/datafiletest' + kcon + '/'
    DatafiletsImgc1 = prostatevalpath + 'seg-eval.txt'
    Imgfiletsc1 = open(DatafiletsImgc1)
    Imgreadc1 = Imgfiletsc1.read().splitlines()
    for Imgnamec1 in Imgreadc1:
        knamelist = Imgnamec1.split("/")
        kname = knamelist[-1][0:6]

        cls0filename = resultpath + '/results/pred_' + kname + 'cls0_prob.nii.gz'
        cls1filename = resultpath + '/results/pred_' + kname + 'cls1_prob.nii.gz'
        cls0read = nib.load(cls0filename)
        cls1read = nib.load(cls1filename)
        cls0logit = cls0read.get_fdata()
        cls1logit = cls1read.get_fdata()
        GTread = nib.load(Imgnamec1)
        GTimg = GTread.get_fdata()
        imgshape = GTimg.shape
        cls0flatten = cls0logit.flatten()
        cls1flatten = cls1logit.flatten()
        clsflatten = np.stack((cls0flatten, cls1flatten))
        GTflatten = GTimg.flatten()
        # By AC
        probflatten = softmax(clsflatten, T = 1.0)
        probr = probflatten.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
        probr_tensor = torch.tensor(probr[np.newaxis, ...])
        GTimgf = np.argmax(probr, axis = 0)
        GT_tensor = torch.tensor(GTimgf[np.newaxis, ...])
        softDSC = SoftDiceLoss(probr_tensor, GT_tensor)
        softDSC_FG_AC.append(softDSC[1].numpy())
        # By ATC
        probflatten = softmax(clsflatten, T = 1.0)
        threshold = 0.877
        probflattens = probflatten > threshold
        probflattens = probflattens.astype(float)
        
        probr = probflatten.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
        GTimgf = np.argmax(probr, axis = 0)
        GT_tensor = torch.tensor(GTimgf[np.newaxis, ...])
        probrs = probflattens.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
        probr_tensor = torch.tensor(probrs[np.newaxis, ...])
        softDSC = SoftDiceLoss(probr_tensor, GT_tensor)
        softDSC_FG_ATC.append(softDSC[1].numpy())
        # By CSATC
        preds_all_argmax = np.argmax(clsflatten, axis = 0)
        # for cls 0, BG class 
        targets_y1 = np.where(preds_all_argmax==0)[0]
        probflatten[:, targets_y1] = softmax(clsflatten[:, targets_y1], T = 1.0)
        threshold = 0.857
        probflattens[:, targets_y1] = probflatten[:, targets_y1] > threshold
        # for cls 1, FG class
        targets_y1 = np.where(preds_all_argmax==1)[0]
        probflatten[:, targets_y1] = softmax(clsflatten[:, targets_y1], T = 1.0)
        threshold = 0.970
        probflattens[:, targets_y1] = probflatten[:, targets_y1] > threshold
        probflattens = probflattens.astype(float)
        probr = probflatten.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
        probrs = probflattens.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
        probr_tensor = torch.tensor(probrs[np.newaxis, ...])
        GTimgf = np.argmax(probr, axis = 0)
        GT_tensor = torch.tensor(GTimgf[np.newaxis, ...])
        softDSC = SoftDiceLoss(probr_tensor, GT_tensor)
        softDSC_FG_CSATC.append(softDSC[1].numpy())

        realDSC, _, _ = ComputMetric(GTimg, np.argmax(probr, axis = 0))
        realDSC_FG.append(realDSC)
        
    softDSCs_AC.append(np.mean(np.array(softDSC_FG_AC)))
    softDSCs_ATC.append(np.mean(np.array(softDSC_FG_ATC)))
    softDSCs_CSATC.append(np.mean(np.array(softDSC_FG_CSATC)))
    realDSCs.append(np.mean(np.array(realDSC_FG)))

100%|█████████████████████| 5/5 [00:39<00:00,  7.97s/it]


In [123]:
print('AC_results:')
print(np.mean(np.abs(np.array(realDSCs) - np.array(softDSCs_AC))))
print('ATC_results:')
print(np.mean(np.abs(np.array(realDSCs) - np.array(softDSCs_ATC))))
print('CSATC_results:')
print(np.mean(np.abs(np.array(realDSCs) - np.array(softDSCs_CSATC))))

AC_results:
1.8734415411981173e-01
ATC_results:
0.16698388786318447
CSATC_results:
0.042532286860843274


In [124]:
print(softDSCs_CSATC)

[0.6378083422909432, 0.697240794087378, 0.6772908320125938, 0.8419924585360163, 0.7973537408007105]


In [125]:
print(softDSCs_ATC)

[0.8467825488872531, 0.8523113325450369, 0.8429700103616687, 0.9247865749636566, 0.8978411492589539]


In [126]:
print(softDSCs_AC)

[0.8903888833710243, 0.8646945640943136, 0.8701191490312503, 0.9339760285393177, 0.9073143222637996]
